In [ ]:
import os
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import re

### Setup

In [ ]:
MONTH = 'April 2019'
FOLDER = 'U:\Data and Analytics\Projects\Data\MOCJ ' + MONTH

In [ ]:
CRIM_NAMES = ["Case#", "Open Date", "Closed Date", "Court", "Latest Event",
              "Intake Type", "TC C/L", "Top Charge", "Dispo"]
POST_DISPO_NAMES = ["Case#", "Open Date", "Closed Date", "Court", "Latest Event",
                    "Intake Type", "TC C/L", "Top Charge", "Dispo", "Case Type"]

MOCJ_CASE_TYPES = CategoricalDtype(categories=["Homicide",
                                               "Violent Felony",
                                               "Non-Violent Felony",
                                               "Misdemeanor",
                                               "Violation & Other"],
                                   ordered=True)
MOCJ_INTAKE_TYPES = CategoricalDtype(categories=["Primary Case", "Conflict Case", "Post Arraignment"],
                                     ordered=True)
MOCJ_OUTFLOW_TYPES = CategoricalDtype(categories=["Post-Arraignment Shift Transfer",
                                                  "Post-Arraignment Disposition",
                                                  "Arraignment Shift Disposition"],
                                      ordered=True)

Functions for classifying top count and intake type, to translate from PDCMS categories to MOCJ categories.

In [ ]:
def classify_intake_type(intake_type):
    """Classifies intake type into MOCJ-requested categories, which are:
    New Cases - Primary Case at Arraignment Shift
    New Cases - Conflict Case at Arraignment Shift
    New Cases - Post Arraignment Shift Other Intake
    """
    intake_type = str(intake_type).strip().lower()
    if (intake_type in ["arraignment", "row"]):
        return ("Primary Case")
    elif (intake_type in ["conflict row", "conflict arraignment"]):
        return ("Conflict Case")
    elif (intake_type.startswith("transfer")) | (intake_type == "assignment"):
        return ("Post Arraignment")
    else:
        return ("DANGER")

In [ ]:
def classify_tc(tc):
    """Classifies top charge class and levels into MOCJ-requested categories.
    """
    tc = str(tc) #coerce TC type to string
    if tc == "H":
        return ("Homicide")
    elif ("FL" in tc) or ("FV" in tc):
        return ("Violent Felony")
    elif (tc.endswith("F") or tc == "F"):
        return ("Non-Violent Felony")
    elif (tc.endswith("M") or tc == "M"):
        return ("Misdemeanor")
    else:
        return ("Violation & Other")

In [ ]:
def categorize_disposition(row):
    if (row["Open Date"] == row["Closed Date"]):
        return "Arraignment Shift Disposition"
    elif (row["Dispo"] in ["RPC", "RLAS", "R18B", "RNDS", "ADBX", "ADBR", "ADQU", "ADRI"]):
        return "Post-Arraignment Shift Transfer"
    else:
        return "Post-Arraignment Shift Disposition"

## Criminal Cases
Criminal cases opened and closed in the month

In [ ]:
crim_cases_opened = pd.read_csv("{}/crim_cases_opened.csv".format(FOLDER),
                                names=CRIM_NAMES,
                                header=0,
                               index_col=False)
crim_cases_closed = pd.read_csv("{}/crim_cases_closed.csv".format(FOLDER),
                                names=CRIM_NAMES,
                                header=0,
                               index_col=False)


crim_cases = pd.concat([crim_cases_opened, crim_cases_closed])
print(crim_cases.shape)

crim_cases = crim_cases.drop_duplicates().reset_index(drop=True)
print(crim_cases.shape)

In [ ]:
crim_cases["Case#"].value_counts()
crim_cases[crim_cases["Case#"] == "21-722054"]

In [ ]:
crim_cases_pending_1st = pd.read_csv("{}/crim_cases_pending_1st.csv".format(FOLDER),
                                     names=CRIM_NAMES, header=0, index_col=False)
crim_cases_pending_end = pd.read_csv("{}/crim_cases_pending_last.csv".format(FOLDER),
                                      names=CRIM_NAMES, header=0, index_col=False)

crim_cases = pd.concat([crim_cases,
                        crim_cases_pending_1st,
                        crim_cases_pending_end])
print(crim_cases.shape)

In [ ]:
crim_cases = crim_cases.dropna(how="all")
print(crim_cases.shape)

crim_cases = crim_cases.drop_duplicates(subset=["Case#", "Open Date", "Closed Date", "Intake Type", "Dispo"],
                                        keep='last').reset_index(drop = True)
print(crim_cases.shape)

# Checking whether a known duplicate has been de-duplicated
crim_cases[crim_cases["Case#"] == '21-709207']

In [ ]:
crim_cases["Open Date"] = pd.to_datetime(crim_cases["Open Date"])
crim_cases["Closed Date"] = pd.to_datetime(crim_cases["Closed Date"])

### Cleaning cases
We don't want non-criminal IDV, VOP, or VOCD cases in these counts, so we're getting rid of cases where the top charge is either VISITATION or FAMILY OFFENSE.

Some top charges have more than one class and level, so we consolidate those.

In [ ]:
# Getting rid of IDV/Visitation cases
crim_cases = crim_cases[~crim_cases['Top Charge'].str.contains("VISITATION", na=False)]
crim_cases = crim_cases[~crim_cases['Top Charge'].str.contains('FAMILY OFFENSE', na=False)]
crim_cases = crim_cases.loc[~((crim_cases["Court"] == "IDV")
                              & (crim_cases["Intake Type"] == "Assignment"))]
crim_cases = crim_cases.loc[~((crim_cases["Court"] == "IDV")
                              & (crim_cases["Intake Type"].str.startswith("Trans")))]

In [ ]:
# Re-code homicide cases to a separate Homicide class, but don't include attempted homicides
# Finds top charges that have "MUR" but don't have "ATT"
crim_cases.loc[crim_cases["Top Charge"].str.contains('MUR', na=False) \
                & ~(crim_cases["Top Charge"].str.contains("ATT", na=False)), "TC C/L"] = "H"
crim_cases.loc[crim_cases["Top Charge"].str.contains("- MUR", na=False), "TC C/L"] = "H"

In [ ]:
crim_cases[crim_cases["TC C/L"] == "H"]

In [ ]:
# Re-code TC C/L for extradition cases
crim_cases.loc[(crim_cases["Top Charge"].str.contains("FUG", na=False)), 'TC C/L'] = "M"

In [ ]:
# AC Misdemeanors don't get counted correctly
crim_cases.loc[(crim_cases["Top Charge"].str.contains("AC - M", na=False)), 'TC C/L'] = "M"

We also check to see if any VOCDs or VOPs have snuck in.

In [ ]:
# Check for any VOCD or VOP cases, then fix in PDCMS until there are no more cases returned
crim_cases[(crim_cases['Top Charge'].str.contains("VIO", na=False))
           & (crim_cases['Top Charge'].str.contains('COND', na=False))]

In [ ]:
# Check for any VOCD or VOP cases, then fix in PDCMS until there are no more cases returned
crim_cases[(crim_cases['Top Charge'].str.contains("VOP", na=False))]

### Determining Start and End

Because this is now a combined dataset, we need to find out whether the cases were opened and/or closed in the month.

In [ ]:
crim_cases[crim_cases["Open Date"] < pd.to_datetime('2019-04-01')]["Open Date"].describe()

In [ ]:
def check_in_month(dt):
    """Check if a date is within the same month and year
    """
    
    month = pd.datetime.today().month -1
    year = pd.datetime.today().year
    return True if (dt.year == year) and (dt.month == month) else False

crim_cases["opened_in_month"] = crim_cases["Open Date"].apply(lambda x: check_in_month(x))
crim_cases["closed_in_month"] = crim_cases["Closed Date"].apply(lambda x: check_in_month(x))

In [ ]:
# Because the dataset is only cases that either opened or closed in the month
# we know that any case not opened in this month will have been pending as of the 1st,
# and we know that any case not closed in this month will have been pending as of the
# last of the month.

crim_cases["pending_first"] = crim_cases["Open Date"].apply(lambda x: True if x < pd.to_datetime('2019-04-01') else False)

In [ ]:
crim_cases["pending_last"] = crim_cases["Closed Date"].apply(
                            lambda x: True if ((x > pd.to_datetime('2019-04-30')) | pd.isnull(x)) else False)

In [ ]:
crim_cases[crim_cases.pending_first == True][["Open Date", "Closed Date"]].describe()

In [ ]:
crim_cases[crim_cases.pending_last == True][["Closed Date", "Open Date"]].describe()

### Classifying Top Charges
Now we translate the top charge type to MOCJ categories and check that everything worked OK.

In [ ]:
#crim_cases["TC C/L"].value_counts()

In [ ]:
crim_cases["TC_Type"] = crim_cases["TC C/L"].apply(lambda x: classify_tc(x))
crim_cases.groupby(["TC_Type", "TC C/L"]).size()

In [ ]:

crim_cases["TC_Type"] = crim_cases.TC_Type.astype(MOCJ_CASE_TYPES)

### Classifying Intake Type

Now we translate the intake type to MOCJ categories and check that everything is OK.

We are assuming that everything with a PDCMS intake type of "Assignment" and a court other than IDV is probably actually a transfer.

In [ ]:
#crim_cases_closed["intake type"] = crim_cases_closed["TC C/L"].apply(lambda x: classify_tc(x))
crim_cases["intake type"] = crim_cases["Intake Type"].apply(lambda x: classify_intake_type(x))

crim_cases["intake type"].value_counts()

In [ ]:
crim_cases[crim_cases["intake type"] == "DANGER"]

In [ ]:
# Make intake type into a Pandas categorical type, so pivot tables are sorted
crim_cases["intake type"] = crim_cases["intake type"].astype(MOCJ_INTAKE_TYPES)

### Classifying Outflow Type

Translate the disposition to MOCJ categories.

In [ ]:
crim_cases["MOCJ_Dispo"] = crim_cases.apply(lambda x: categorize_disposition(x), axis=1)
crim_cases.groupby(["MOCJ_Dispo", "Dispo"]).size()

In [ ]:
crim_cases.groupby(["MOCJ_Dispo"]).size()

### Final Aggregate Counts

In [ ]:
crim_cases[crim_cases.opened_in_month == True].pivot_table(values=["Case#"],
                                                                index=["TC_Type", "intake type"],
                                                                aggfunc=np.count_nonzero)

In [ ]:
crim_cases[crim_cases.closed_in_month == True].pivot_table(values=["Case#"],
                                                           index=["TC_Type", "MOCJ_Dispo"],
                                                           aggfunc=np.count_nonzero)

In [ ]:
crim_cases[crim_cases.pending_first == True].pivot_table(values=["Case#"], index=["TC_Type"],
                                                         aggfunc=np.count_nonzero)

In [ ]:
crim_cases[crim_cases.pending_last == True].pivot_table(values=["Case#"], index=["TC_Type"],
                                                        aggfunc=np.count_nonzero)

In [ ]:
crim_cases["case_age"] = (crim_cases["Closed Date"] - crim_cases["Open Date"]).dt.days
crim_cases.case_age.describe()

In [ ]:
crim_cases[crim_cases.closed_in_month == True].groupby('TC_Type').agg({'case_age': 'describe'})

In [ ]:
crim_cases[crim_cases.case_age > 200][["Case#", "Court", "Latest Event", "Top Charge", "Dispo", "case_age"]]

## Post-Disposition Cases

In [ ]:
pd_cases = pd.concat([pd.read_csv("{}/post_dispo_opened.csv".format(FOLDER),
                                  names=POST_DISPO_NAMES,
                                  header=0),
                      pd.read_csv("{}/post_dispo_closed.csv".format(FOLDER),
                                  names=POST_DISPO_NAMES,
                                  header=0),
                      pd.read_csv("{}/post_dispo_pending_1st.csv".format(FOLDER),
                                  names=POST_DISPO_NAMES,
                                  header=0),
                      pd.read_csv("{}/post_dispo_pending_last.csv".format(FOLDER),
                                  names=POST_DISPO_NAMES,
                                  header=0)])

pd_cases = pd_cases.drop_duplicates(subset=["Case#", "Open Date", "Closed Date"],
                                    keep='last').reset_index(drop = True)
pd_cases = pd_cases.dropna(how='all')

pd_cases["Case Type"] = pd_cases["Case Type"].str.upper()

Create dummy variables for whether the case was opened and/or closed in the month.

In [ ]:
pd_cases["Open Date"] = pd.to_datetime(pd_cases["Open Date"])
pd_cases["Closed Date"] = pd.to_datetime(pd_cases["Closed Date"])
pd_cases["opened_in_month"] = pd_cases["Open Date"].apply(lambda x: check_in_month(x))
pd_cases["closed_in_month"] = pd_cases["Closed Date"].apply(lambda x: check_in_month(x))

Create a dummy variable for pending on the 1st of the month or the last.

In [ ]:
pd_cases["pending_first"] = pd_cases["Open Date"].apply(lambda x: True if x < pd.to_datetime('2019-04-01') else False)

In [ ]:
pd_cases["pending_last"] = pd_cases["Closed Date"].apply(
                            lambda x: True if ((x > pd.to_datetime('2019-04-30')) | pd.isnull(x)) else False)

In [ ]:
pd_cases[pd_cases.pending_first == True].groupby("Case Type").size()

In [ ]:
pd_cases[pd_cases.opened_in_month == True].groupby("Case Type").size()

In [ ]:
pd_cases[pd_cases.closed_in_month == True].groupby("Case Type").size()

In [ ]:
pd_cases[pd_cases.pending_last == True].groupby("Case Type").size()